In [3]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np
import re
from tqdm import tqdm
import matplotlib.pyplot as plt

# Define the age-conditional Generator
class AgeControlGenerator(nn.Module):
    def __init__(self, input_channels=3, output_channels=3, age_dim=16, ngf=64):
        super(AgeControlGenerator, self).__init__()

        # Age embedding - convert scalar age to a feature vector
        self.age_embedding = nn.Sequential(
            nn.Linear(1, age_dim),
            nn.ReLU(),
            nn.Linear(age_dim, age_dim),
            nn.ReLU()
        )

        # Initial convolutional layer
        self.init_conv = nn.Sequential(
            nn.Conv2d(input_channels, ngf, kernel_size=7, stride=1, padding=3),
            nn.InstanceNorm2d(ngf),
            nn.ReLU(inplace=True)
        )

        # Downsampling layers
        self.down1 = self._make_down_block(ngf, ngf*2)
        self.down2 = self._make_down_block(ngf*2, ngf*4)

        # Residual blocks with age condition
        self.res_blocks = nn.ModuleList([
            self._make_res_block(ngf*4) for _ in range(9)
        ])

        # Age condition blocks - inject age information into the network
        self.age_condition1 = self._make_age_condition_block(ngf*4, age_dim)
        self.age_condition2 = self._make_age_condition_block(ngf*4, age_dim)
        self.age_condition3 = self._make_age_condition_block(ngf*4, age_dim)

        # Upsampling layers
        self.up1 = self._make_up_block(ngf*4, ngf*2)
        self.up2 = self._make_up_block(ngf*2, ngf)

        # Output layer
        self.output_conv = nn.Sequential(
            nn.Conv2d(ngf, output_channels, kernel_size=7, stride=1, padding=3),
            nn.Tanh()
        )

    def _make_down_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=2, padding=1),
            nn.InstanceNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def _make_res_block(self, channels):
        return nn.Sequential(
            nn.Conv2d(channels, channels, kernel_size=3, stride=1, padding=1),
            nn.InstanceNorm2d(channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(channels, channels, kernel_size=3, stride=1, padding=1),
            nn.InstanceNorm2d(channels)
        )

    def _make_age_condition_block(self, channels, age_dim):
        return nn.Sequential(
            nn.Conv2d(channels + age_dim, channels, kernel_size=3, stride=1, padding=1),
            nn.InstanceNorm2d(channels),
            nn.ReLU(inplace=True)
        )

    def _make_up_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.InstanceNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x, age):
        # Normalize age to [0, 1] range (assuming age is between 0-100)
        normalized_age = age / 100.0

        # Get age embeddings
        age_embed = self.age_embedding(normalized_age.view(-1, 1))  # [B, age_dim]

        # Initial convolution
        x = self.init_conv(x)

        # Downsampling
        x = self.down1(x)
        x = self.down2(x)

        # Apply first 3 residual blocks
        for i in range(3):
            res = self.res_blocks[i](x)
            x = x + res

        # Inject first age condition
        batch_size, _, h, w = x.size()
        age_spatial = age_embed.view(batch_size, -1, 1, 1).expand(-1, -1, h, w)
        x_with_age = torch.cat([x, age_spatial], dim=1)
        x = self.age_condition1(x_with_age) + x

        # Apply middle 3 residual blocks
        for i in range(3, 6):
            res = self.res_blocks[i](x)
            x = x + res

        # Inject second age condition
        age_spatial = age_embed.view(batch_size, -1, 1, 1).expand(-1, -1, h, w)
        x_with_age = torch.cat([x, age_spatial], dim=1)
        x = self.age_condition2(x_with_age) + x

        # Apply final 3 residual blocks
        for i in range(6, 9):
            res = self.res_blocks[i](x)
            x = x + res

        # Inject third age condition
        age_spatial = age_embed.view(batch_size, -1, 1, 1).expand(-1, -1, h, w)
        x_with_age = torch.cat([x, age_spatial], dim=1)
        x = self.age_condition3(x_with_age) + x

        # Upsampling
        x = self.up1(x)
        x = self.up2(x)

        # Output layer
        x = self.output_conv(x)

        return x

# Define the age-conditional Discriminator
class AgeControlDiscriminator(nn.Module):
    def __init__(self, input_channels=3, ndf=64):
        super(AgeControlDiscriminator, self).__init__()

        # Age embedding
        self.age_embedding = nn.Sequential(
            nn.Linear(1, ndf),
            nn.LeakyReLU(0.2, inplace=True)
        )

        # Initial layers
        self.init_conv = nn.Sequential(
            nn.Conv2d(input_channels, ndf, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True)
        )

        # Downsampling layers
        self.down1 = nn.Sequential(
            nn.Conv2d(ndf + 1, ndf * 2, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True)
        )

        self.down2 = nn.Sequential(
            nn.Conv2d(ndf * 2, ndf * 4, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True)
        )

        self.down3 = nn.Sequential(
            nn.Conv2d(ndf * 4, ndf * 8, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True)
        )

        # Output layer
        self.output_layer = nn.Conv2d(ndf * 8, 1, kernel_size=4, stride=1, padding=1)

    def forward(self, x, age):
        # Normalize age to [0, 1] range
        normalized_age = age / 100.0

        # Initial conv
        h = self.init_conv(x)

        # Create age condition maps
        batch_size, _, height, width = h.size()
        age_map = normalized_age.view(batch_size, 1, 1, 1).expand(batch_size, 1, height, width)

        # Concatenate feature maps with age condition
        h = torch.cat([h, age_map], dim=1)

        # Downsampling
        h = self.down1(h)
        h = self.down2(h)
        h = self.down3(h)

        # Output layer
        output = self.output_layer(h)

        return output

# Custom Dataset for face images with age from filename
class AgeControlDataset(Dataset):
    def __init__(self, root_dir, transform=None, age_pattern=r'(\d+)_\d+_\d+_\d+\.jpg$'):
        """
        Dataset for face images with age extraction from filename

        Args:
            root_dir: Directory with all the images
            transform: Optional transform to be applied on images
            age_pattern: Regex pattern to extract age from filename
                         Default pattern matches files like 'older_28_1_2_20170116162739672.jpg'
        """
        self.root_dir = root_dir
        self.transform = transform
        self.age_pattern = age_pattern

        # Get all image files
        self.image_files = []

        for filename in os.listdir(root_dir):
            if filename.endswith(('.jpg', '.jpeg', '.png')):
                # Extract age from filename
                age_match = re.search(self.age_pattern, filename)
                if age_match:
                    age = int(age_match.group(1))
                    self.image_files.append((filename, age))

        print(f"Found {len(self.image_files)} valid images with age information.")

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name, age = self.image_files[idx]
        img_path = os.path.join(self.root_dir, img_name)

        # Load image
        image = Image.open(img_path).convert('RGB')

        # Apply transforms
        if self.transform:
            image = self.transform(image)

        # Convert age to tensor
        age_tensor = torch.tensor(age, dtype=torch.float32)

        return image, age_tensor

# Function to train the age-conditional GAN
def train_age_control_gan(data_dir, output_dir, num_epochs=100, batch_size=16,
                          lr=0.0002, beta1=0.5, beta2=0.999, image_size=256,
                          age_pattern=r'(\d+)_\d+_\d+_\d+\.jpg$', device="cuda"):
    """
    Train the age-conditional GAN

    Args:
        data_dir: Directory containing training images
        output_dir: Directory to save model checkpoints and samples
        num_epochs: Number of training epochs
        batch_size: Batch size for training
        lr: Learning rate
        beta1, beta2: Adam optimizer betas
        image_size: Size of input/output images
        age_pattern: Regex pattern to extract age from filename
        device: Device to use for training ('cuda' or 'cpu')
    """
    # Create output directories
    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(os.path.join(output_dir, "checkpoints"), exist_ok=True)
    os.makedirs(os.path.join(output_dir, "samples"), exist_ok=True)

    # Set device
    device = torch.device(device if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Define transforms
    transform = transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    # Create dataset and dataloader
    dataset = AgeControlDataset(data_dir, transform=transform, age_pattern=age_pattern)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4)

    # Initialize models
    generator = AgeControlGenerator().to(device)
    discriminator = AgeControlDiscriminator().to(device)

    # Initialize optimizers
    optimizer_G = optim.Adam(generator.parameters(), lr=lr, betas=(beta1, beta2))
    optimizer_D = optim.Adam(discriminator.parameters(), lr=lr, betas=(beta1, beta2))

    # Loss functions
    adversarial_loss = nn.BCEWithLogitsLoss()
    l1_loss = nn.L1Loss()

    # Sample fixed images and ages for visualization
    fixed_dataloader = DataLoader(dataset, batch_size=4, shuffle=True)
    fixed_images, fixed_ages = next(iter(fixed_dataloader))
    fixed_images = fixed_images.to(device)
    fixed_ages = fixed_ages.to(device)

    # Generate target ages for fixed images (10, 30, 50, 70, 90)
    fixed_target_ages = torch.tensor([10, 30, 50, 70, 90], dtype=torch.float32, device=device)

    # Training loop
    total_steps = 0
    for epoch in range(num_epochs):
        for i, (images, ages) in enumerate(tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}")):
            # Move data to device
            real_images = images.to(device)
            real_ages = ages.to(device)

            # Get batch size
            batch_size = real_images.size(0)

            # Generate random target ages for the batch (10-90)
            target_ages = torch.randint(10, 91, (batch_size,), device=device).float()

            # ---------------------
            # Train Discriminator
            # ---------------------

            optimizer_D.zero_grad()

            # Real images with real ages - should be classified as real
            real_validity = discriminator(real_images, real_ages)
            real_label = torch.ones_like(real_validity, device=device)
            loss_real = adversarial_loss(real_validity, real_label)

            # Generated images with target ages - should be classified as fake
            fake_images = generator(real_images, target_ages)
            fake_validity = discriminator(fake_images.detach(), target_ages)
            fake_label = torch.zeros_like(fake_validity, device=device)
            loss_fake = adversarial_loss(fake_validity, fake_label)

            # Total discriminator loss
            d_loss = (loss_real + loss_fake) * 0.5
            d_loss.backward()
            optimizer_D.step()

            # ---------------------
            # Train Generator
            # ---------------------

            optimizer_G.zero_grad()

            # Try to fool the discriminator
            fake_validity = discriminator(fake_images, target_ages)
            g_loss_adv = adversarial_loss(fake_validity, real_label)

            # Identity loss (when target age = source age)
            # More robust approach for identity loss calculation
            same_age_mask = (target_ages - real_ages).abs() < 5
            if same_age_mask.sum() > 0:
              g_loss_id = l1_loss(
               fake_images[same_age_mask],
              real_images[same_age_mask]
               ) * 10.0
            else:
               g_loss_id = torch.tensor(0.0, device=device)

            # Age transformation cycle consistency
            cycled_images = generator(fake_images, real_ages)
            g_loss_cycle = l1_loss(cycled_images, real_images) * 10.0

            # Total generator loss
            g_loss = g_loss_adv + g_loss_id + g_loss_cycle
            g_loss.backward()
            optimizer_G.step()

            # Print progress
            if i % 50 == 0:
                print(f"[Epoch {epoch+1}/{num_epochs}] [Batch {i}/{len(dataloader)}] "
                      f"[D loss: {d_loss.item():.4f}] [G loss: {g_loss.item():.4f} "
                      f"(adv: {g_loss_adv.item():.4f}, id: {g_loss_id.item():.4f}, "
                      f"cycle: {g_loss_cycle.item():.4f})]")

            total_steps += 1

        # Save model checkpoints
        if (epoch + 1) % 10 == 0 or (epoch + 1) == num_epochs:
            torch.save(generator.state_dict(),
                      os.path.join(output_dir, "checkpoints", f"generator_epoch_{epoch+1}.pth"))
            torch.save(discriminator.state_dict(),
                      os.path.join(output_dir, "checkpoints", f"discriminator_epoch_{epoch+1}.pth"))

        # Generate and save sample images
        if (epoch + 1) % 5 == 0 or (epoch + 1) == 1:
            # Create samples directory for this epoch
            epoch_samples_dir = os.path.join(output_dir, "samples", f"epoch_{epoch+1}")
            os.makedirs(epoch_samples_dir, exist_ok=True)

            # Set models to evaluation mode
            generator.eval()

            with torch.no_grad():
                # Generate images for each fixed image at different target ages
                for img_idx, (image, age) in enumerate(zip(fixed_images, fixed_ages)):
                    image = image.unsqueeze(0)  # Add batch dimension

                    # Create visualization grid
                    fig, axes = plt.subplots(1, len(fixed_target_ages) + 1, figsize=(3*(len(fixed_target_ages) + 1), 3))

                    # Original image
                    orig_img = (image.squeeze(0).cpu() * 0.5 + 0.5).numpy().transpose(1, 2, 0)
                    axes[0].imshow(np.clip(orig_img, 0, 1))
                    axes[0].set_title(f"Original: {age.item():.0f}y")
                    axes[0].axis("off")

                    # Generate images for each target age
                    for age_idx, target_age in enumerate(fixed_target_ages):
                        # Generate aged image
                        aged_image = generator(image, target_age.unsqueeze(0))

                        # Convert to numpy for visualization
                        aged_img = (aged_image.squeeze(0).cpu() * 0.5 + 0.5).numpy().transpose(1, 2, 0)

                        # Display in grid
                        axes[age_idx + 1].imshow(np.clip(aged_img, 0, 1))
                        axes[age_idx + 1].set_title(f"Age: {target_age.item():.0f}y")
                        axes[age_idx + 1].axis("off")

                    # Save figure
                    plt.tight_layout()
                    plt.savefig(os.path.join(epoch_samples_dir, f"sample_{img_idx}.png"))
                    plt.close()

            # Set models back to training mode
            generator.train()

    # Save final models
    torch.save(generator.state_dict(), os.path.join(output_dir, "checkpoints", "generator_final.pth"))
    torch.save(discriminator.state_dict(), os.path.join(output_dir, "checkpoints", "discriminator_final.pth"))

    print(f"Training completed. Models saved to {output_dir}/checkpoints")

# Function to generate age-transformed images
def generate_age_transformed_faces(model_path, input_dir, output_dir, ages=None,
                                  age_pattern=r'(\d+)_\d+_\d+_\d+\.jpg$', device="cuda"):
    """
    Generate age-transformed versions of input faces

    Args:
        model_path: Path to the trained generator model
        input_dir: Directory containing input images
        output_dir: Directory to save generated images
        ages: List of target ages (default: 10, 20, ..., 90)
        age_pattern: Regex pattern to extract age from filename
        device: Device to use for inference
    """
    if ages is None:
        ages = list(range(10, 91, 10))  # 10, 20, 30, ..., 90

    # Create output directory
    os.makedirs(output_dir, exist_ok=True)

    # Load model
    device = torch.device(device if torch.cuda.is_available() else "cpu")
    generator = AgeControlGenerator().to(device)
    generator.load_state_dict(torch.load(model_path, map_location=device))
    generator.eval()

    # Define transform
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    # Process each image in the input directory
    for filename in tqdm(os.listdir(input_dir), desc="Generating age-transformed faces"):
        if not filename.endswith(('.jpg', '.jpeg', '.png')):
            continue

        # Extract age from filename
        age_match = re.search(age_pattern, filename)
        if age_match:
            original_age = int(age_match.group(1))
        else:
            print(f"Could not extract age from {filename}, skipping...")
            continue

        # Load and preprocess image
        img_path = os.path.join(input_dir, filename)
        image = Image.open(img_path).convert('RGB')
        image_tensor = transform(image).unsqueeze(0).to(device)

        # Create output subfolder for this image
        base_filename = os.path.splitext(filename)[0]
        image_output_dir = os.path.join(output_dir, base_filename)
        os.makedirs(image_output_dir, exist_ok=True)

        # Create grid for comparison
        num_ages = len(ages)
        grid_width = num_ages + 1  # original + all target ages
        grid = Image.new('RGB', (256 * grid_width, 256))

        # Add original image to grid
        grid.paste(image.resize((256, 256)), (0, 0))

        # Generate image for each target age
        with torch.no_grad():
            for i, target_age in enumerate(ages):
                # Convert target age to tensor
                age_tensor = torch.tensor([target_age], dtype=torch.float32, device=device)

                # Generate aged image
                aged_image_tensor = generator(image_tensor, age_tensor)

                # Convert to PIL image
                aged_image = transforms.ToPILImage()(aged_image_tensor.squeeze(0).cpu() * 0.5 + 0.5)

                # Save individual image
                aged_image.save(os.path.join(image_output_dir, f"age_{target_age}.jpg"))

                # Add to grid
                grid.paste(aged_image, ((i + 1) * 256, 0))

        # Save grid
        grid.save(os.path.join(image_output_dir, "age_progression.jpg"))

        # Save metadata
        with open(os.path.join(image_output_dir, "metadata.txt"), 'w') as f:
            f.write(f"Original image: {filename}\n")
            f.write(f"Original age: {original_age}\n")
            f.write(f"Generated ages: {', '.join(map(str, ages))}\n")

# Main function
def main():
    # Paths
    data_dir = "/content/drive/MyDrive/UTKFace/cGAN_train"  # Directory with training images
    model_dir = "/content/drive/MyDrive/age_control_gan"  # Directory to save models
    test_dir = "/content/drive/MyDrive/UTKFace/cGAN_test"  # Directory with test images
    output_dir = "/content/drive/MyDrive/UTKFace/age_transformed"  # Directory for output images

    # Define age pattern regex for your filenames
    # For example, for "older_28_1_2_20170116162739672.jpg", the pattern extracts "28"
    age_pattern = r'older_(\d+)_\d+_\d+_\d+\.jpg$'

    # Check if trained model exists
    model_path = os.path.join(model_dir, "checkpoints", "generator_final.pth")

    if not os.path.exists(model_path):
        print("Training new age control GAN model...")
        train_age_control_gan(
    data_dir=data_dir,
    output_dir=model_dir,
    age_pattern=r'older_(\d+)_\d+_\d+_\d+\.jpg\.chip\.jpg$',  # Updated pattern
    num_epochs=100,
    batch_size=16
)

    # Generate age-transformed faces
    print("Generating age-transformed faces...")
    generate_age_transformed_faces(
    model_path=model_path,
    input_dir=test_dir,
    output_dir=output_dir,
    ages=[10, 20, 30, 40, 50, 60, 70, 80, 90],
    age_pattern=r'older_(\d+)_\d+_\d+_\d+\.jpg\.chip\.jpg$'  # Updated pattern
)

    print("Done!")

if __name__ == "__main__":
    # Set random seed for reproducibility
    torch.manual_seed(42)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(42)

    main()

Training new age control GAN model...
Using device: cuda
Found 808 valid images with age information.


Epoch 1/100:   2%|▏         | 1/51 [00:09<08:09,  9.79s/it]

[Epoch 1/100] [Batch 0/51] [D loss: 0.7053] [G loss: 11.4629 (adv: 0.7366, id: 5.3034, cycle: 5.4228)]


Epoch 1/100: 100%|██████████| 51/51 [01:20<00:00,  1.58s/it]

[Epoch 1/100] [Batch 50/51] [D loss: 0.4395] [G loss: 6.7985 (adv: 1.7446, id: 2.2950, cycle: 2.7589)]


...

Epoch 100/100: 100%|██████████| 51/51 [00:58<00:00,  1.15s/it]

[Epoch 100/100] [Batch 50/51] [D loss: 0.0046] [G loss: 7.4333 (adv: 6.5817, id: 0.0000, cycle: 0.8515)]


Training completed. Models saved to /content/drive/MyDrive/age_control_gan/checkpoints
Generating age-transformed faces...


Generating age-transformed faces: 100%|██████████| 215/215 [00:34<00:00,  6.27it/s]

Done!


In [2]:
import shutil
import os

# Remove the directory if it exists
if os.path.exists('/content/drive'):
    shutil.rmtree('/content/drive')

# Create a new empty directory
os.makedirs('/content/drive')

# Now try mounting again
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Mount Google Drive to access the dataset
from google.colab import drive
drive.mount('/content/drive')

import os
import random
import shutil
import re
import collections
import numpy as np
from sklearn.model_selection import train_test_split

def extract_age_from_filename(filename):
    """Extract age information from filename using regex pattern for UTKFace dataset."""
    # Pattern to extract age from "older_XX_" format
    pattern = r'older_(\d+)_'
    match = re.search(pattern, filename)
    if match:
        return int(match.group(1))
    return None

def split_dataset_age_balanced(source_dir, train_dir, test_dir, train_ratio=0.8, seed=42):
    """
    Splits images into train and test sets with age-balanced distribution.
    All files are assumed to be in the source_dir (flat structure).

    Parameters:
    - source_dir: str, path to the source dataset directory
    - train_dir: str, path where the training set will be saved
    - test_dir: str, path where the testing set will be saved
    - train_ratio: float, ratio of images to place in the training set (Default 0.8)
    - seed: int, random seed for reproducibility
    """
    # Set seed for reproducibility
    random.seed(seed)
    np.random.seed(seed)

    # Create output directories
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    # Dictionary to store images by age
    age_to_images = collections.defaultdict(list)

    # Get all image files
    all_files = [f for f in os.listdir(source_dir)
                if os.path.isfile(os.path.join(source_dir, f)) and
                f.lower().endswith(('.jpg', '.jpeg', '.png', '.jpg.chip.jpg'))]

    print(f"Found {len(all_files)} image files in source directory")

    # Group images by age
    for img in all_files:
        img_path = os.path.join(source_dir, img)
        age = extract_age_from_filename(img)

        if age is not None:
            age_to_images[age].append((img_path, img))
        else:
            print(f"Warning: Could not extract age from filename: {img}")

    # For each age group, split into train and test while maintaining the train_ratio
    age_stats = {"train": collections.Counter(), "test": collections.Counter()}

    for age, images in age_to_images.items():
        # Shuffle images of this age
        random.shuffle(images)

        # Split images based on train_ratio
        split_idx = int(len(images) * train_ratio)
        train_images = images[:split_idx]
        test_images = images[split_idx:]

        # Copy train images
        for src_path, img_name in train_images:
            dst_path = os.path.join(train_dir, img_name)
            shutil.copy2(src_path, dst_path)
            age_stats["train"][age] += 1

        # Copy test images
        for src_path, img_name in test_images:
            dst_path = os.path.join(test_dir, img_name)
            shutil.copy2(src_path, dst_path)
            age_stats["test"][age] += 1

    # Print statistics
    print("\nDataset split complete!")
    print(f"Train set: {sum(age_stats['train'].values())} images")
    print(f"Test set: {sum(age_stats['test'].values())} images")

    print("\nAge distribution in train set:")
    for age in sorted(age_stats["train"].keys()):
        print(f"Age {age}: {age_stats['train'][age]} images ({age_stats['train'][age]/sum(age_stats['train'].values())*100:.1f}%)")

    print("\nAge distribution in test set:")
    for age in sorted(age_stats["test"].keys()):
        print(f"Age {age}: {age_stats['test'][age]} images ({age_stats['test'][age]/sum(age_stats['test'].values())*100:.1f}%)")

    # Calculate and print ratio similarity to verify balance
    print("\nVerifying age distribution balance between train and test sets:")
    all_ages = sorted(set(list(age_stats["train"].keys()) + list(age_stats["test"].keys())))

    train_total = sum(age_stats["train"].values())
    test_total = sum(age_stats["test"].values())

    for age in all_ages:
        train_pct = (age_stats["train"][age] / train_total) * 100 if train_total > 0 else 0
        test_pct = (age_stats["test"][age] / test_total) * 100 if test_total > 0 else 0
        diff = abs(train_pct - test_pct)

        print(f"Age {age}: Train {train_pct:.1f}% vs Test {test_pct:.1f}% (Difference: {diff:.1f}%)")

# Example usage
if __name__ == "__main__":
    # Update these paths to match your Google Drive structure
    source_dir = '/content/drive/MyDrive/UTKFace/10_29_test_out'  # Your current folder with images
    train_dir = '/content/drive/MyDrive/UTKFace/cGAN_train'       # Destination for train set
    test_dir  = '/content/drive/MyDrive/UTKFace/cGAN_test'        # Destination for test set

    split_dataset_age_balanced(source_dir, train_dir, test_dir, train_ratio=0.8, seed=42)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 1023 image files in source directory

Dataset split complete!
Train set: 808 images
Test set: 215 images

Age distribution in train set:
Age 10: 14 images (1.7%)
Age 11: 4 images (0.5%)
Age 12: 10 images (1.2%)
Age 13: 8 images (1.0%)
Age 14: 12 images (1.5%)
Age 15: 16 images (2.0%)
Age 16: 19 images (2.4%)
Age 17: 12 images (1.5%)
Age 18: 32 images (4.0%)
Age 19: 8 images (1.0%)
Age 20: 29 images (3.6%)
Age 21: 35 images (4.3%)
Age 22: 39 images (4.8%)
Age 23: 44 images (5.4%)
Age 24: 91 images (11.3%)
Age 25: 70 images (8.7%)
Age 26: 200 images (24.8%)
Age 27: 44 images (5.4%)
Age 28: 68 images (8.4%)
Age 29: 53 images (6.6%)

Age distribution in test set:
Age 10: 4 images (1.9%)
Age 11: 2 images (0.9%)
Age 12: 3 images (1.4%)
Age 13: 3 images (1.4%)
Age 14: 4 images (1.9%)
Age 15: 5 images (2.3%)
Age 16: 5 images (2.3%)
Age 17: 4 images (1.9%)
Age 1